In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing as pp
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_sim.csv", index_col=0)
df

In [ ]:
df2 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_dummy.csv", index_col=0)
df2

In [ ]:
df3 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_rank.csv", index_col=0)
df3

In [ ]:
def df_no_words(df):
    df.reset_index(inplace=True)
    idk = df.copy()
    idk.set_index(['user', 'label_id'], inplace=True)
    idk.rename(columns={'rank': 'top'}, inplace=True)
    
    return idk


In [ ]:
ready = df_no_words(df3)

In [ ]:
import random
index2 = random.sample(range(152), 30)

In [ ]:
index2[0]

In [ ]:
ready.xs(index2[0], level='user', drop_level=False)

In [ ]:
test_set2 = pd.DataFrame()
for i in range(len(index2)):
    test_set2 = test_set2.append(ready.xs(index2[i], level='user', drop_level=False))


In [ ]:
test_set2

In [ ]:
train_set2 = ready.copy()

In [ ]:
train_set2

In [ ]:
train_set2.reset_index(level=1, inplace=True)

In [ ]:
for i in range(len(index2)):
    train_set2 = train_set2.drop(train_set2.loc[(train_set2.index == index2[i])].index)

In [ ]:
train_set2.set_index('label_id', append=True, inplace=True)


In [ ]:
train_set2

In [ ]:
train_set = train_set2.drop(columns=['city'])
test_set = test_set2.drop(columns=['city'])

In [ ]:
sm = SMOTE(random_state = 33)
x_train, x_test, y_train, y_test = train_test_split(ready.drop('top', axis=1),
                                                    ready.top, test_size= 0.2, random_state=12)


In [ ]:
type(y_train)

In [ ]:
sm = SMOTE(random_state = 33)

X_train_new, y_train_new = sm.fit_sample(x_train, y_train.ravel())

In [ ]:
x_train

In [ ]:
train_set['top']

In [ ]:

y_train_set = train_set['top']
train_set.drop(columns=['top'], inplace=True)


In [ ]:
y_test_set = test_set['top']
test_set.drop(columns=['top'], inplace=True)

In [ ]:
x_test = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_test.csv')
y_test = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_test.csv')

x_train = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_train.csv')
y_train = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_train.csv')

In [ ]:
y_train.set_index(['user', 'label_id'], inplace=True)
x_train.set_index(['user', 'label_id'], inplace=True)
y_test.set_index(['user', 'label_id'], inplace=True)
x_test.set_index(['user', 'label_id'], inplace=True)


In [ ]:
y_train

In [ ]:
y_trains = pd.Series(y_train['top'].values, index=y_train.index)


In [ ]:
y_trains

In [ ]:
sm = SMOTE(random_state = 33)


X_train_new, y_train_new = sm.fit_sample(x_train, y_trains.ravel())

In [ ]:
nr = NearMiss()
X_train_NM, y_train_NM = nr.fit_sample(x_train, y_train)

In [ ]:
scaled_features_df

In [ ]:
x_train

### Logistic Regression

Ranks lbfgs  
Accuary is .97
But precision and recall, f1-score for 1 = 0

#### SMOTE lbfgs
P: .07  
R= .27  
f1=  .11

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', max_iter=10000).fit(X_train_new, y_train_new)
predictions_lr = clf.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_lr))
print(confusion_matrix(y_test,predictions_lr))


### KNN


#### Rank
precision: .12  
recall: .01  
f1: .01  

#### SMOTE
P:  .08  
R:  .28  
f1: .12  

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
neigh.fit(X_train_new, y_train_new)
predictions_kn = neigh.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_kn))
print(confusion_matrix(y_test,predictions_kn))


### SVM


#### Rank
0 for all - rbf

#### SMOTE
rbf
P= .09  
R= .64  
f1 = .15  


In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf')
svclassifier.fit(x_train_new, y_train_new)
predictions_svm = svclassifier.predict(x_test)


In [ ]:
print(confusion_matrix(y_test,predictions_svm))
print(classification_report(y_test,predictions_svm))

In [ ]:
test.top.value_counts()

### Decision Tree


#### rank
precision: .25  
recall: .26  
f1: .25  


#### SMOTE
P= .20  
R= 26  
F1= .22  

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train, y_train)
predictions_dt = clf.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_dt))
print(confusion_matrix(y_test,predictions_dt))


### Random Forest


#### Rank
Precision: .94  
recall: .10  
f1:  .19


#### SMOTE
P= .83  
R= .13  
F1= .23  

In [ ]:
#'max_depth': 25, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} random_state=12

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=12, max_depth=25)
clf.fit(X_train_new, y_train_new)
predictions_rf = clf.predict(x_test)
probs_rf = clf.predict_proba(x_test)

In [ ]:
predictions_train = clf.predict(x_train)
probs_train = clf.predict_proba(x_train)

In [ ]:
x_train.shape

In [ ]:
print(classification_report(y_train, predictions_train))
print(confusion_matrix(y_train,predictions_train))

In [ ]:
print(classification_report(y_test, predictions_rf))
print(confusion_matrix(y_test,predictions_rf))


In [ ]:
probs_train

In [ ]:
y_train_set

In [ ]:
clf.get_params()

In [ ]:
x_train_sm.columns

In [ ]:
rf_sm = RandomForestClassifier(random_state=12, max_depth=25)
rf_sm.fit(x_train_sm, y_train_sm)
predictions_rf_sm = rf_sm.predict(x_test)
probs_rf_sm = rf_sm.predict_proba(x_test)

In [ ]:
print(classification_report(y_test_set, predictions_rf_sm))
print(confusion_matrix(y_test_set,predictions_rf_sm))

### Creating accuracy metric

In [ ]:
results5 = x_test.copy()

In [ ]:
results3

In [ ]:
def split_array(a, df, predict):
    lst1 = list(a[:, 0])
    lst2 = list(a[:, 1])
    predict_lst = list(predict)
    df['prob1_group'] = pd.Series(lst1, index=df.index)
    df['prob2_group'] = pd.Series(lst2, index=df.index)
    df['predicted'] = pd.Series(predict_lst, index=df.index)
    
    df = add_city(df)
    
    return df

In [ ]:
def add_city(df):
    city_df = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_rank.csv", index_col=0)
    
    # adds actual top ranked cities
    df = pd.merge(right=y_test, left=df, right_index=True, left_index=True)
    
    city_df.reset_index(inplace=True)
    city = city_df[['user', 'label_id', 'city']].copy()
    city.set_index(['user', 'label_id'], inplace=True)
    # Adds city names
    df = pd.merge(right=df, left=city, left_index=True, right_index=True, how='right')
    
    return df


In [ ]:
results5 = split_array(probs_rf, results5, predictions_rf)

In [ ]:
results5[results5['predicted'] == 1]

In [ ]:
# Create dataframe of highest 9 prob2 score for each user 
test_set_top = results5.groupby('user').apply(lambda x: x.nlargest(5, 'prob2_group')).droplevel(level=0)

In [ ]:
test_set_top[test_set_top['top'] == 1]

In [ ]:
sim_score = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/suprise_weight.csv", index_col=0)


In [ ]:
sim_score['city'] = sim_score['city'].str.replace('Kraków', 'Krakow')
sim_score['city'] = sim_score['city'].str.replace('Muğla', 'Mugla')
sim_score['city'] = sim_score['city'].str.replace('São Paulo', 'Sao Paulo')
sim_score['city'] = sim_score['city'].str.replace('Zürich', 'Zurich')
sim_score['city'] = sim_score['city'].str.replace('Düsseldorf', 'Dusseldorf')
sim_score['city'] = sim_score['city'].str.replace('Frankfurt am Main', 'Frankfurt')

In [ ]:
results5

In [ ]:
results5.reset_index(inplace=True)

In [ ]:
results5.user.unique()

In [ ]:
finished = pd.merge(right=results5, left=sim_score, on=['user', 'city'], how='right')


In [ ]:
accuracy = finished[['user', 'city', 'predicted', 'top', 'ranking_weight', 'prob2_group', 'prob1_group']]

In [ ]:
accuracy.set_index(['user', 'city'], inplace=True)

In [ ]:
accuracy.sort_values('ranking_weight')

* Create dataframe without the .5 added for top cities  
Merge into this one.  

select out top 5 for each user with high prob2 group - give a 1-5 rank  

select out top 5 for each user with high ranking_weight  

Is any 1-5 ranked prob2 group also a ranked 1-5 with ranking weight. count how many



### Scrap work

In [ ]:
results_top = results.groupby('user').apply(lambda x: x.nlargest(9, 'prob2_group')).droplevel(level=0)

In [ ]:
results_top.xs(109, level='user', drop_level=False)

In [ ]:
test_set_new2[test_set_new2['top'] == 1]

In [ ]:
test_set_top.city.value_counts()

In [ ]:

train_set_full = split_array(probs_train, train_set, predictions_train)
train_set_full

In [ ]:
train_set_full = pd.merge(right=train_set2, left=y_train_set, right_index=True, left_index=True)
train_set_full = pd.merge(right=train_set2, left=train_set2['city'], right_index=True, left_index=True)
train_set_top = train_set2.groupby('user').apply(lambda x: x.nlargest(9, 'prob2_group')).droplevel(level=0)


In [ ]:
user22 = split_array(one_probs, new_user)
user22

In [ ]:
sorted_user_group = user2.sort_values('prob2_group', ascending=False)
sorted_user = user2.sort_values('prob2', ascending=False)

In [ ]:
sorted_user.head(10)

In [ ]:
sorted_user_group.head(10)

In [ ]:
sorted_user[sorted_user['rank'] == 1]

In [ ]:
unine

In [ ]:
predictions_rf

In [ ]:
y_test

In [ ]:
probs_rf

In [ ]:
lst1 = list(probs_rf[:, 0])

In [ ]:
lst2 = list(probs_rf[:, 1])

In [ ]:
probs_rf[:, 1]

In [ ]:
df_y_test = pd.DataFrame(y_test)
df_y_test

In [ ]:
df_y_test['prob_new'] = pd.Series(prob, index=df_y_test.index)

In [ ]:
df_y_test['prob_1'] = pd.Series(lst1, index=df_y_test.index)
df_y_test['prob_2'] = pd.Series(lst2, index=df_y_test.index)

In [ ]:
df_y_test.sort_values('prob_2', ascending=False)

In [ ]:
df_y_test[df_y_test['prob_2'] > .47]

In [ ]:
predict_lst = list(predictions_rf)
df_y_test['predict'] = pd.Series(predict_lst, index=df_y_test.index)


In [ ]:
L = list(data)
A = list(data)
data_frame = pd.DataFrame(L) 
data_frame['A'] = pd.Series(A, index=data_frame.index)

In [ ]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(clf, hyperF, cv = 2, verbose = 3, 
                      n_jobs = -1)
bestF = gridF.fit(X_train_new, y_train_new)

In [ ]:
predictions_rfcv = bestF.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_rfcv))
print(confusion_matrix(y_test,predictions_rf))


bestF.best_params_

{'max_depth': 25,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}
 random_state=12
 
 
 P= .66  
 R= .13  
 F1 = .22  
 
 
 
 4074  4  
 126   19  

In [ ]:
clf.get_params()

### Adding continents to city

In [ ]:
df3

In [ ]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2


In [ ]:
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}

cities['continent'] = [continents[country_alpha2_to_continent_code(country_name_to_country_alpha2(country))] for country in cities['country']]

In [ ]:
import sqlite3


In [ ]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
cities = get_df('cities')

cities

In [ ]:

cities.continent.value_counts()

In [ ]:
df_city = df3['city']
df_city